In [27]:
import sys
import os
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER')
import fastText
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import *
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
from keras.callbacks import ModelCheckpoint
import math

from keras.utils import to_categorical
import linecache
from keras.callbacks import Callback

epochs = 50
trainable=True

In [4]:
nb_sequence_length=50
nb_embedding_dims=300

# To load the initial model for transfer learning

In [5]:
def createMatrices(sentences, word2Idx, case2Idx, char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
        charIndices = []
#         labelIndices = []
        
        for word,char in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
#             labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices,caseIndices, charIndices]) 
        
    return dataset


def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences

In [142]:
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')
print(caseEmbeddings.shape)

(8, 8)


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [144]:
characters={}

with open ('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_one_million') as f:
    for line in f:
        for word in line:
            word =str(word)
            for char in word:
#                 print(char)
                characters[char]=True

In [145]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)

In [146]:
toy_vocab ={}

with open('twitter_dataset_sequences_one_million') as f:
    for line in f:
#         print(line)
        line = line.split()
#         print(len(line))
        for word in line:
#             word = word
#             print(word)
            toy_vocab[word]=True

In [147]:
word2Idx={}
for word in toy_vocab.keys():
    word2Idx[word] = len(word2Idx)

In [148]:
vocab_size = len(word2Idx.keys())+1
print(vocab_size)

917425


In [149]:
# for wiki
words_input = Input(shape=(None,300), dtype='float32',name='words_input')
# words = Embedding(input_dim =50, output_dim=300, trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words_input, char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.5))(output)
# output = TimeDistributed(Dense(vocab_size))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
# output = Flatten()(output)
output = Dense(vocab_size, activation='softmax')(output)
# output = Flatten()(output)
# output = Dense(vocab_size, activation='softmax')(output)
# model.add(Dense(vocab_size, activation='softmax'))
model = Model(inputs=[words_input,casing_input, character_input], outputs=[output])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
# model.summary()

In [8]:
# final model
model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
lstm_block = LeakyReLU()(lstm_block)

filter_sizes = (3, 4, 5)
conv_blocks = []
for sz in filter_sizes:
    conv = Conv1D(
        filters = 200,
        kernel_size = sz,
        padding = 'valid',
        strides = 1
    )(lstm_block)
    conv = LeakyReLU()(conv)
    conv = GlobalMaxPooling1D()(conv)
    conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
# model_concatenated = Dropout(0.8)(model_concatenated)
model_concatenated = Dense(100)(model_concatenated)
model_concatenated = LeakyReLU()(model_concatenated)
model_output = Dense(300, activation = "softmax")(model_concatenated)
model = Model(model_input_embedding, model_output)
model.compile(loss='cosine_proximity', optimizer='nadam', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 50, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 50, 200)      320800      input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 50, 200)      0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 48, 200)      120200      leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
conv1d_2 (

In [9]:
model.load_weights('german_lm_twitter_50m_1.h5')

# Doing Transfer Learning

In [10]:
for layer in model.layers:
    print(layer.name)

input_2
bidirectional_2
leaky_re_lu_1
conv1d_1
conv1d_2
conv1d_3
leaky_re_lu_2
leaky_re_lu_3
leaky_re_lu_4
global_max_pooling1d_1
global_max_pooling1d_2
global_max_pooling1d_3
dropout_1
dropout_2
dropout_3
concatenate_1
dense_1
leaky_re_lu_5
dense_2


In [11]:
model.layers.pop()

In [12]:
for layer in model.layers:
    print(layer.name)

input_2
bidirectional_2
leaky_re_lu_1
conv1d_1
conv1d_2
conv1d_3
leaky_re_lu_2
leaky_re_lu_3
leaky_re_lu_4
global_max_pooling1d_1
global_max_pooling1d_2
global_max_pooling1d_3
dropout_1
dropout_2
dropout_3
concatenate_1
dense_1
leaky_re_lu_5


In [13]:
characters={}
labels = {}
# with open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training.txt') as f:
with open('/home/gwiedemann/notebooks/OffLang/sample_train.txt') as f:
    for line in f:
        text, label_simple, label_complex = line.split('\t')
        labels[label_simple]=True
        for word in text:
            for char in word:
                characters[char]=True
                

In [14]:
label2Idx = {}
for label in labels:
    label2Idx[label] = len(label2Idx)

In [15]:
print(label2Idx)

{'OFFENSE': 0, 'OTHER': 1}


In [16]:
output = Dense(len(label2Idx), activation = 'softmax')(model.layers[-1].output)

In [17]:
final_model = Model(inputs=model.input, outputs=[output])

In [18]:
for layer in final_model.layers:
    print(layer.name)

input_2
bidirectional_2
leaky_re_lu_1
conv1d_1
conv1d_2
conv1d_3
leaky_re_lu_2
leaky_re_lu_3
leaky_re_lu_4
global_max_pooling1d_1
global_max_pooling1d_2
global_max_pooling1d_3
dropout_1
dropout_2
dropout_3
concatenate_1
dense_1
leaky_re_lu_5
dense_3


In [19]:
final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
final_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 50, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 50, 200)      320800      input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 50, 200)      0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 48, 200)      120200      leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
conv1d_2 (

In [103]:
out_file1 = open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_simple_labels.txt','wb')
out_file2 = open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_complex_labels.txt','wb')
with open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training.txt', ) as f:
    for line in f:
        line = line.split('\t')
#         print(len(line))
        text, label_simple, label_complex = line
        simple = [text, label_simple]
        compl = [text, label_complex]
        string_simple = '\t'.join(simple)+' \n'
        string_complex = '\t'.join(compl)+' \n'
        out_file1.write(string_simple.encode())
        out_file2.write(string_complex.encode())
out_file1.close()
out_file2.close()

In [162]:
line_number=0

In [20]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [164]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)

In [165]:
# def file_len(fname):
#     with open(fname) as f:
#         for i, l in enumerate(f):
#             pass
#     return i + 1

In [33]:
def sequential_generator(filename, batch_size):
    
    file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
    shuffled_indexes = range(1, file_length + 1)
    # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
    index_position = 0
    
    batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims))
    # batch_features_idx = np.zeros((batch_size, nb_sequence_length))
    batch_labels = np.zeros((batch_size, 2))

    while True:
        # print(len(features))
        for i in range(batch_size):
            line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            batch_features_ft[i], batch_features_lg[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            index_position += 1
            if index_position == file_length:
                # shuffle indexes again
                shuffled_indexes = range(1, file_length + 1)
                # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
                index_position = 0
                break
        # yield [batch_features_ft, batch_features_lg], batch_labels
        yield [batch_features_ft], batch_labels

In [29]:
line_number=0
for inp, output in my_generator('/home/gwiedemann/notebooks/OffLang/sample_train.txt',32):
    continue
#     print(inp[0].shape)
#     print(inp[1].shape)
#     print(inp[2].shape)
#     print(output.shape)

KeyboardInterrupt: 

In [168]:
# def validation_data_generator(file, batch_size):
    
#     global line_number_validation, lock

#     while True:
#         word_embeddings = []
#         case_embeddings = []
#         char_embeddings = []
#         output_labels = []
#         total_lines = file_len(file)
#         for i in range(batch_size):
            
# #             lock.acquire()
            
#             index = line_number_validation%total_lines +1
#             line_number_validation+=1
# #             lock.release()
# #             index = random.choice(len(features), 1)[0]
#             line = linecache.getline(file, index)
#             if len(line.split('\t'))!=3:
#                 continue
#             text, simple_label, complex_label = line.split('\t')
# #             assert index<=100
# #             print(text)
#             temp_casing = []
#             temp_char=[]
#             temp_word=[]
# #             print(line)
# #             print(len)
#             for word in text.split():
# #                 print(word)
#                 casing =getCasing(word, case2Idx)
# #                 print(casing)
#                 temp_casing.append(casing)
#                 temp_char2=[]
#                 for char in word:
#                     temp_char2.append(char2Idx[char])
#                 temp_char2 = np.array(temp_char2)
# #                 print(temp_char2)
# #                 temp_char2 = pad_sequences(temp_char2, 52, padding='post')
# #                 temp_char.append(pad_sequences(temp_char2, 52, padding='post'))
#                 temp_char.append(temp_char2)
#                 word_vector = ft.get_word_vector(word.lower())
# #                 print(len(word_vector))
#                 temp_word.append(word_vector)
#             temp_char = pad_sequences(temp_char, 52)
# #             print(temp_word)
# #             print(len(temp_word))
# #             print(len(temp_casing))
# #             print(temp_char)
# #             print(len(temp_char))
# #             print(label2Idx[label])
# #             print(index)
# #             print(" **************** ")
#             word_embeddings.append(temp_word)
# #             print((len(word_embeddings), len(word_embeddings[0]), len(word_embeddings[0][0])))

#             case_embeddings.append(temp_casing)
#             char_embeddings.append(temp_char)
#             output_labels.append(label2Idx[simple_label])
# #             print(np.array(word_embeddings))
# #             batch_features[i] = process_features(line, window_size-1, nb_embedding_dims)
#             # print(batch_features[i])
#             # print(batch_features[i].shape)
# #             batch_labels[i] = labels[index]
#         yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], to_categorical(output_labels, len(label2Idx)))
            

In [24]:
train_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_train.txt', encoding = "UTF-8")]
dev_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_dev.txt', encoding = "UTF-8")]

In [25]:
train_sentences = [x[0] for x in train_lines]
train_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in train_lines])
# train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

dev_sentences = [x[0] for x in dev_lines]
dev_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in dev_lines])
# dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

In [28]:
samples_per_epoch = len(train_sentences)
epochs = 50
batch_size = 32
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
checkpoint = ModelCheckpoint('best_classification_model.h5', 
                             monitor='val_acc', 
                             verbose = 1, 
                             save_best_only = True, 
                             save_weights_only = True)

In [29]:
n_labels = 2

In [34]:
dev_batch_size = len(dev_sentences)
final_model.fit_generator(
    sequential_generator('/home/gwiedemann/notebooks/OffLang/sample_train.txt', batch_size), 
    steps_per_epoch=steps_per_epoch, epochs=epochs,
    validation_data = sequential_generator('/home/gwiedemann/notebooks/OffLang/sample_dev.txt', dev_batch_size),
    validation_steps = math.ceil(len(dev_sentences) / dev_batch_size),
    callbacks = [checkpoint]
)
final_model.reset_states()

Epoch 1/50


NameError: name 'process_features' is not defined

In [192]:
idx2Label={}
for key, value in label2Idx.items():
    idx2Label[value]=key

In [193]:
def return_predictions(filename, model):
    gold_file = open('/home/jindal/notebooks/jindal/NER/language_model/gold_pred_simple.txt','w')
    pred_file = open('/home/jindal/notebooks/jindal/NER/language_model/pred_simple.txt','w')
    true_labels=[]
    pred_labels=[]
    with open(filename) as f:
        for line in f:
#             print(line.split('\t')[0], end=' ')
            word_embeddings=[]
            case_embeddings=[]
            char_embeddings=[]
            if len(line.split('\t'))!=3:
                    continue
            text, simple_label, complex_label = line.split('\t')
            to_write_gold = text+'\t'+simple_label+'\t'+simple_label+'\n'
            to_write_pred = text
            temp_casing = []
            temp_char=[]
            temp_word=[]
            for word in text.split():
                casing =getCasing(word, case2Idx)
                temp_casing.append(casing)
                temp_char2=[]
                for char in word:
                    temp_char2.append(char2Idx[char])
                temp_char2 = np.array(temp_char2)
                temp_char.append(temp_char2)
                word_vector = ft.get_word_vector(word.lower())
                temp_word.append(word_vector)
            temp_char = pad_sequences(temp_char, 52)
            word_embeddings.append(temp_word)
            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
    #         output_labels.append(label2Idx[label_simple])
            inp= [np.array([temp_word]), np.array([temp_casing]), np.array([temp_char])]
    #         yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], to_categorical(output_labels, len(label2Idx)))
    #         print(final_model.predict_classes(inp))
            prediction = model.predict(inp,batch_size=1)[0].argmax(-1)
            to_write_pred = to_write_pred + '\t'+ idx2Label[prediction]+'\t'+ idx2Label[prediction]+'\n'
            correct = label2Idx[simple_label]
#             print(prediction,correct)
            gold_file.write(to_write_gold)
            pred_file.write(to_write_pred)
            pred_labels.append(prediction)
            true_labels.append(correct)
#             print(idx2Label[final_model.predict(inp, batch_size=1)[0].argmax(-1)])
    #         y_classes = y_prob.argmax(axis=-1)
        gold_file.close()
        pred_file.close()
        return pred_labels, true_labels


In [194]:
final_model.load_weights('model_with_pretraining.h5')

In [195]:
pred , correct = return_predictions('/home/gwiedemann/notebooks/OffLang/sample_dev.txt',final_model)

sklearn.metrics.f1_score(correct, pred, average='macro')

0.7266464364799474

# Making model without pretraining to compare the results

In [196]:
words_input = Input(shape=(None,300), dtype='float32',name='words_input')
# words = Embedding(input_dim =50, output_dim=300, trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words_input, char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.5))(output)
# output = TimeDistributed(Dense(vocab_size))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
# output = Flatten()(output)
output = Dense(len(label2Idx), activation = 'softmax')(output)
# output = Flatten()(output)
# output = Dense(vocab_size, activation='softmax')(output)
# model.add(Dense(vocab_size, activation='softmax'))
model_without_pre = Model(inputs=[words_input,casing_input, character_input], outputs=[output])
model_without_pre.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
# model.summary()

In [ ]:
line_number=0
line_number_validation=0
# training_file_name = '/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_simple_labels.txt'
training_file_name = '/home/gwiedemann/notebooks/OffLang/sample_train.txt'
dev_file_name = '/home/gwiedemann/notebooks/OffLang/sample_dev.txt'
checkpoint = ModelCheckpoint('model_no_pretrain.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_without_pre.fit_generator(my_generator(training_file_name, 1)
                         , epochs =30 , steps_per_epoch = file_len(training_file_name)//1, validation_data=validation_data_generator(dev_file_name, 1),
                         validation_steps=807, callbacks=[checkpoint])

Epoch 1/30
4199/4200 [============================>.] - ETA: 0s - loss: 0.6346 - acc: 0.6621
Epoch 00001: val_acc improved from -inf to 0.71747, saving model to model_no_pretrain.h5
4200/4200 [==============================] - 435s 104ms/step - loss: 0.6348 - acc: 0.6619 - val_loss: 0.5662 - val_acc: 0.7175
Epoch 2/30
4199/4200 [============================>.] - ETA: 0s - loss: 0.5695 - acc: 0.7123
Epoch 00002: val_acc improved from 0.71747 to 0.73234, saving model to model_no_pretrain.h5
4200/4200 [==============================] - 428s 102ms/step - loss: 0.5695 - acc: 0.7124 - val_loss: 0.5208 - val_acc: 0.7323
Epoch 3/30
4199/4200 [============================>.] - ETA: 0s - loss: 0.5247 - acc: 0.7440
Epoch 00003: val_acc improved from 0.73234 to 0.76828, saving model to model_no_pretrain.h5
4200/4200 [==============================] - 432s 103ms/step - loss: 0.5249 - acc: 0.7438 - val_loss: 0.4968 - val_acc: 0.7683
Epoch 4/30
4199/4200 [============================>.] - ETA: 0s - l

In [198]:
pred , correct = return_predictions('/home/gwiedemann/notebooks/OffLang/sample_dev.txt',model_without_pre)

sklearn.metrics.f1_score(correct, pred, average='binary')

0.6511627906976744

In [124]:
print(label2Idx)

{'OTHER': 0, 'OFFENSE': 1}
